# Trim the Sample Arxiv 2018 Corpus

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
import copy
import editdistance
import matplotlib.pyplot as plt
import gensim
import os
import pandas as pd
import re
import sys
import time

sys.path.append('../../')
import tmx

## Dictionary and Corpus

In [ ]:
C_ROOT = '../../corpora/arxiv/arxiv-2018'

### Dictionary

In [ ]:
dct = tmx.utils.unpickle(os.path.join(C_ROOT, 'arxiv-2018.mm.dictionary.cpickle'))

Most-used tokens:

In [ ]:
tokens = pd.DataFrame([
    {
        'token': dct[t],
        'docs': dct.dfs[t],
        'docs_frac': dct.dfs[t]/dct.num_docs
    } 
    for t in dct
])

In [ ]:
tokens = tokens.sort_values('docs', ascending=False)

In [ ]:
tokens.head()

### Corpus

In [ ]:
arxiv = gensim.corpora.MmCorpus(os.path.join(C_ROOT, 'arxiv-2018.mm'))

## Keep a Selection of Tokens

Mostly arbitrary, but following the online LDA paper:

  - Remove anything in over 85% of documents
  - Take the top 20,000 tokens
  - Merge similar tokens
  - Use resulting top 7,500 for LDA

In [ ]:
# Remove most frequent tokens
tokens = tokens[tokens['docs_frac'] < 0.85]

In [ ]:
# Top 20,000
tokens = tokens.iloc[:20000,:]

In [ ]:
# Top 7,500 unique
# Merge tokens that differ by 1 character (s, d, or punctuation) at the end
# Merge unpopular tokens into popular ones
token_list = list(tokens['token'])
RE_ENDING = re.compile(r'([sd]|[^a-zA-Z0-9])$')

merged = {}
i = 0
time_start = time_delta = time.time()
while len(set(merged.values())) <= 7500:
    i += 1

    # Current token
    token = token_list.pop(0)
    merged[token] = token
    
    # Similar tokens
    similar_tokens = [
        t for t in token_list
        if editdistance.distance(token, t) <= 1
        and t.startswith(token)
        and (RE_ENDING.search(t) is not None or 
             RE_ENDING.search(token) is not None)
    ]
    for t in similar_tokens:
        #print(f'Merged: {t} => {token}')
        merged[t] = token
        token_list.remove(t)
    
    if i%100 == 0:
        print('Status: {:>5d}  {:>5d}  {:>5d}  {:>5.2f}  {:>7.2f}'.format(
            i, len(set(merged.keys())), len(set(merged.values())), 
            time.time()-time_delta, time.time()-time_start))
        time_delta = time.time()

## Generate New Data Sets

### New Dictionary

See, e.g., [this Q&A](https://github.com/RaRe-Technologies/gensim/wiki/recipes-&-faq#q8-how-can-i-filter-a-saved-corpus-and-its-corresponding-dictionary) for how to truncate a dictionary _and_ corpus together.

In [ ]:
old_dct = dct
new_dct = copy.deepcopy(old_dct)

In [ ]:
# Only keep the final tokens from 'merged'
new_dct.filter_tokens(good_ids={old_dct.token2id[token] for token in merged.values()})

In [ ]:
len(new_dct)

In [ ]:
new_dct.save(os.path.join(C_ROOT, 'arxiv-2018-trunc.mm.dictionary.cpickle'))

### New Corpus

In [ ]:
# Make a mapping from orig ID <-> new ID after the merge
old_to_new = {
    old_dct.token2id[old_token]: new_dct.token2id[new_token]
    for old_token, new_token in merged.items()
}

In [ ]:
vt = gensim.models.VocabTransform(old_to_new)

In [ ]:
gensim.corpora.MmCorpus.serialize(os.path.join(C_ROOT, 'arxiv-2018-trunc.mm'), vt[arxiv])